# LangChain 1.0: Agents, HITL, and Multi-Agent Systems

Welcome to this guide on leveraging LangChain 1.0! In this notebook, we'll explore the modern way to build agents using the `create_agent` workflow.

We will cover:
1. **Simple Agents**: Building a basic agent with tools.
2. **Human-in-the-Loop (HITL)**: Adding oversight to our agents.
3. **Retrieval**: Giving our agent access to external data (RAG) using OpenAI embeddings and Qdrant.
4. **Multi-Agent Systems**: Composing multiple agents together.

All the core logic is mirrored in the `src/` directory, demonstrating how to structure your project for deployment with LangSmith.


## 1. Setup and Installation
Let's get our environment ready by `uv sync`ing!

## 2. Environment Variables
We need to set our OpenAI API Key. We also enable LangSmith tracing here.

In [1]:
import os
import getpass

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

_set_if_undefined("OPENAI_API_KEY")

# LangSmith Tracing
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "LangChain 1.0 Guide"
_set_if_undefined("LANGSMITH_API_KEY")


## 3. The `create_agent` Workflow

LangChain 1.0 introduces `create_agent` as the standard way to build agents. It simplifies the process while retaining the power of LangGraph under the hood.

We've defined a simple agent in `src/agent.py` that has access to a weather tool and a "magic calculator".


In [2]:
from src.agent import build_simple_agent

# Build the agent
agent = build_simple_agent()

# Run the agent
response = agent.invoke({"messages": [{"role": "user", "content": "What is the weather in San Francisco?"}]})
print(response["messages"][-1].content)

USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/chris/Code/AI Makerspace/Events/LangChain1.0/.venv/lib/python3.13/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


The weather in San Francisco is currently rainy.


## 4. Human-in-the-Loop (HITL)

Sometimes we want to approve sensitive actions before they happen. LangChain 1.0 makes this easy with middleware.

We've configured our `hitl_agent` to interrupt before using the `magic_calculator` tool.


In [5]:
from src.agent import build_hitl_agent
from langgraph.types import Command
# Import uuid7 for generating valid thread IDs compliant with LangSmith
from langsmith import uuid7

# Build the HITL agent
hitl_agent = build_hitl_agent()

# Configuration for the thread (required for checkpointing)
# Using uuid7 to avoid warnings and ensure best practice
thread_id = str(uuid7())
config = {"configurable": {"thread_id": thread_id}}

print(f"--- Asking to calculate (Thread: {thread_id}) ---")
# We use .stream() to see steps
events = list(hitl_agent.stream(
    {"messages": [{"role": "user", "content": "Please use the magic calculator to add 5 and 5."}]},
    config=config
))

# Print events to verify the initial run and interruption
for i, event in enumerate(events):
    # print(f"Event {i} keys: {event.keys()}")
    if "messages" in event:
        event["messages"][-1].pretty_print()

--- Asking to calculate (Thread: 019a9d46-3d3b-7704-a29a-d553acb69ee3) ---


You should see the tool call request, and then the stream ends (due to interruption). We can now resume.

In [6]:
# Resume execution
# We need to construct the resume payload dynamically based on the interrupt ID.

# 1. Fetch the current state
state = hitl_agent.get_state(config)

# 2. Find the interrupt
tasks = state.tasks
resume_payload = {}

if tasks and tasks[0].interrupts:
    interrupt = tasks[0].interrupts[0]
    print(f"Found interrupt: {interrupt.id}")
    
    # 3. Construct payload mapping interrupt ID to decision
    resume_payload = {
        interrupt.id: {
            "decisions": [{"type": "approve"}]
        }
    }
else:
    print("No active interrupts found. Agent might have finished.")

# 4. Resume if we have a payload
if resume_payload:
    print(f"--- Resuming (Thread: {thread_id}) ---")
    resume_output = list(hitl_agent.stream(
        Command(resume=resume_payload),
        config=config
    ))
    
    if not resume_output:
        print("No events received after resume. Checking final state...")
        final_state = hitl_agent.get_state(config)
        if final_state.values and "messages" in final_state.values:
             final_state.values["messages"][-1].pretty_print()
    else:
        for event in resume_output:
            if "messages" in event:
                event["messages"][-1].pretty_print()
            else:
                print(event)

Found interrupt: a1ca32f1062ebba96b965ab156c80e2e
--- Resuming (Thread: 019a9d46-3d3b-7704-a29a-d553acb69ee3) ---
{'HumanInTheLoopMiddleware.after_model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 112, 'total_tokens': 131, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_b1442291a8', 'id': 'chatcmpl-CdglVpeNLfx6QKf0NOQMX88qIBN2c', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--03f19c29-5a17-4c52-9105-c9143b85a048-0', tool_calls=[{'name': 'magic_calculator', 'args': {'a': 5, 'b': 5}, 'id': 'call_jdj09EAqk9Q15zNTBIdnadOF', 'type': 'tool_call'}], usage_metadata={'input_tokens': 112, 'output_tokens': 

## 5. Retrieval (RAG)

Agents become truly powerful when they can access external knowledge. We can add a retriever as a tool.

We've set up a RAG pipeline over the "Musk v Altman" complaint in `src/rag.py`, using OpenAI embeddings and Qdrant.


In [8]:
from src.agent import build_rag_agent

rag_agent = build_rag_agent()

response = rag_agent.invoke({
    "messages": [{"role": "user", "content": "When should I use LangChain 1.0?"}]
})

print(response["messages"][-1].content)

I couldn't find specific information on when to use LangChain 1.0. However, LangChain is generally used for building applications with language models, and version 1.0 likely includes updates and features that enhance its capabilities. If you have specific requirements or features in mind, I can help you find more detailed information.


## 6. Multi-Agent Systems

LangChain 1.0 agents are graphs, which means they can be composed! We can treat an agent as a tool for another agent.

We have a "Supervisor" that delegates to a "Researcher" (who has RAG access) and a "Writer" (who has a specific persona).


In [9]:
from src.agent import build_multi_agent_system

supervisor = build_multi_agent_system()

# This complex query requires research and then writing
query = "When should I use LangChain vs. LangGraph, write a Shakespearean sonnet about that."

response = supervisor.invoke({
    "messages": [{"role": "user", "content": query}]
})

print(response["messages"][-1].content)

Here is a Shakespearean sonnet about when to use LangChain versus LangGraph:

When language flows like rivers swift and bright,  
And words must dance upon the tongue with grace,  
Then LangChain's art shall guide thee through the night,  
To craft a voice that time cannot erase.  

Yet when the mind seeks webs of thought profound,  
Where data's threads in complex patterns weave,  
LangGraph shall rise, its structured paths unbound,  
To map the truths that hidden depths conceive.  

LangChain, the bard of chat and speech refined,  
In simple verse, it speaks to hearts and minds.  
LangGraph, the sage of knowledge intertwined,  
In graphs and nodes, the deeper meaning finds.  

Choose LangChain for the art of language pure,  
For graphs and depth, let LangGraph be your cure.
